Import needed libraries

In [6]:
import pandas as pd

Read data

In [7]:
labeled_df = pd.read_feather('hf://datasets/pfacouetey/DSTI_Deep_Learning_Project_2025/labeled_data.feather')

'(ReadTimeoutError("HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 0fd867f3-55f2-4713-9755-4772b122ba95)')' thrown while requesting GET https://huggingface.co/datasets/pfacouetey/DSTI_Deep_Learning_Project_2025/resolve/main/labeled_data.feather
Retrying in 1s [Retry 1/5].


In [8]:
labeled_df.head(n=3)

,text,label
0,dear american teens question dutch person hear...,0
1,nothing look forward lifei dont many reasons k...,1
2,music recommendations im looking expand playli...,0


In [9]:
unlabeled_df = pd.read_feather('hf://datasets/pfacouetey/DSTI_Deep_Learning_Project_2025/unlabeled_data.feather')

'(ReadTimeoutError("HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 170f5c59-2a76-4947-abd2-64fbd00e28e8)')' thrown while requesting GET https://huggingface.co/datasets/pfacouetey/DSTI_Deep_Learning_Project_2025/resolve/main/unlabeled_data.feather
Retrying in 1s [Retry 1/5].


In [10]:
unlabeled_df.head(n=3)

,text
0,Any idea what this is? So I came here for awns...
1,"Advice, please. I've been getting so easily en..."
2,Can aggressive behavior in humans be modified ...
